## Dự đoán chất lượng rượu vang

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
# cung cấp các tiện ích cho mở rộng, chuyển đổi và xử lý.
# utilities for scaling, transforming, and wrangling data.
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
# pipeline
from sklearn.pipeline import make_pipeline
# cross-validation(CV)
from sklearn.model_selection import GridSearchCV
# đánh giá hiệu suất mô hình(evaluate model performance)
from sklearn.metrics import mean_squared_error, r2_score
# duy trì mô hình cho tương lai(persist model for future)
# joblib là sự thay thế cho gói pickle, hiệu quả hơn để lưu trữ các mảng numpy lớn
# more efficient for storing large numpy arrays.
from sklearn.externals import joblib

In [2]:
# read csv from URL
dataset_url = 'http://mlr.cs.umass.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
# tệp tin CSV sử dụng dấu chấm phẩy để tách dữ liệu
# CSV file using semicolons to separate data.
data = pd.read_csv(dataset_url, sep=';')
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [3]:
data.shape

(1599, 12)

In [4]:
data.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [5]:
y = data.quality # target
X = data.drop('quality', axis=1)
# 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=123, 
                                                    stratify=y)

In [6]:
# Sử dụng API chuyển đổi, tính toán trên tập huấn luyện, sau đó lưu lại mean và độ lệch chuẩn để dùng cho các chuyển đổi sau.
# Use the transformer API, computing on training set, mean and standard deviation are then stored 
# to be used on later data using the transform method.
scaler = preprocessing.StandardScaler().fit(X_train)
# Áp dụng chuyển đổi cho traning set
# Apply the transformer to the training set
X_train_scaled = scaler.transform(X_train)
X_train_scaled.mean(axis=0)

array([  1.16664562e-16,  -3.05550043e-17,  -8.47206937e-17,
        -2.22218213e-17,   2.22218213e-17,  -6.38877362e-17,
        -4.16659149e-18,  -2.54439854e-15,  -8.70817622e-16,
        -4.08325966e-16,  -1.17220107e-15])

In [7]:
X_train_scaled.std(axis=0)

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [8]:
# chuyển đổi tập kiểm tra sử dụng cùng giá trị mean và độ lệch chuẩn đã được sử dụng cho tập huấn luyện trước đó
# transform the test set using the same means and standard deviations used to transform the training set:
X_test_scaled = scaler.transform(X_test)
X_test_scaled.mean(axis=0)

array([ 0.02776704,  0.02592492, -0.03078587, -0.03137977, -0.00471876,
       -0.04413827, -0.02414174, -0.00293273, -0.00467444, -0.10894663,
        0.01043391])

In [9]:
X_test_scaled.std(axis=0)

array([ 1.02160495,  1.00135689,  0.97456598,  0.91099054,  0.86716698,
        0.94193125,  1.03673213,  1.03145119,  0.95734849,  0.83829505,
        1.0286218 ])

Lưu ý rằng các tính năng được scale trong tập kiểm tra không hoàn toàn tập trung ở mức 0. Đây chính xác là những gì ta mong đợi, vì chúng ta đang chuyển đổi tập kiểm tra sử dụng các phương tiện từ tập huấn luyện, chứ không phải từ tập kiểm tra.

In [10]:
# một mô hình ống dẫn chuyển đổi dữ liệu bằng cách sử dụng StandardScaler () và sau đó fit với Random Forest Regressor model
# a modeling pipeline transforms the data using StandardScaler() and then fits a model using a random forest regressor.
pipeline = make_pipeline(preprocessing.StandardScaler(), 
                         RandomForestRegressor(n_estimators=100))

In [11]:
# Liệt kê các siêu tham số có thể điều chỉnh
# list the tunable hyperparameters
pipeline.get_params()

{'memory': None,
 'randomforestregressor': RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
            max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0, warm_start=False),
 'randomforestregressor__bootstrap': True,
 'randomforestregressor__criterion': 'mse',
 'randomforestregressor__max_depth': None,
 'randomforestregressor__max_features': 'auto',
 'randomforestregressor__max_leaf_nodes': None,
 'randomforestregressor__min_impurity_decrease': 0.0,
 'randomforestregressor__min_impurity_split': None,
 'randomforestregressor__min_samples_leaf': 1,
 'randomforestregressor__min_samples_split': 2,
 'randomforestregressor__min_weight_fraction_leaf': 0.0,
 'randomforestregressor__n_estimators': 100,
 'randomforestregressor__n_jobs': 1,
 

In [12]:
# Điều chỉnh tham số như sau:
# tune hyperparameter like so:
hyperparameters = { 'randomforestregressor__max_features' : ['auto', 'sqrt', 'log2'],
                  'randomforestregressor__max_depth': [None, 10, 5, 3, 1]}

Đạt hiệu suất cao hơn nếu sử dụng cross-validation pipeline

In [13]:
# cross-validation pipeline
clf = GridSearchCV(pipeline, hyperparameters, cv=10)
# Fit and tune model
clf.fit(X_train, y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('randomforestregressor', RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decr...mators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'randomforestregressor__max_features': ['auto', 'sqrt', 'log2'], 'randomforestregressor__max_depth': [None, 10, 5, 3, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [14]:
clf.best_params_

{'randomforestregressor__max_depth': None,
 'randomforestregressor__max_features': 'log2'}

Sau khi điều chỉnh các hyperparameter thích hợp bằng cách sử dụng cross-validation, ta thường có thể cải thiện hiệu suất bằng cách sửa lại mô hình trên toàn bộ tập huấn luyện. Thuận lợi là GridSearchCV đã tự động làm điều này

In [15]:
# kiểm tra có đang bật tự động refit ko
clf.refit

True

In [16]:
# dự đoán trên tập dữ liệu mới
# predict a new set of data:
y_pred = clf.predict(X_test)

In [17]:
# đánh giá hiệu suất mô hình: sử dụng r2_score và mean_squared_error
# evaluate model performance
print(r2_score(y_test, y_pred))
print(mean_squared_error(y_test, y_pred))

0.465259156123
0.3450540625


In [18]:
# Lưu mô hình để sử dụng trong tương lai
# Save model for future use
joblib.dump(clf, 'rf_regressor.pkl')

['rf_regressor.pkl']

In [19]:
# Khi muốn dùng lại mô hình để dự đoán
# load the model again to predict
clf2 = joblib.load('rf_regressor.pkl')
clf2.predict(X_test)

array([ 6.35,  5.65,  4.96,  5.48,  6.32,  5.63,  5.14,  4.75,  5.  ,
        6.03,  5.24,  5.7 ,  5.9 ,  5.14,  5.82,  5.69,  6.6 ,  5.81,
        5.68,  6.98,  5.47,  5.61,  5.03,  6.06,  5.91,  5.03,  5.52,
        5.17,  5.79,  5.97,  5.86,  6.53,  5.98,  5.05,  4.99,  5.96,
        5.04,  6.1 ,  5.05,  6.03,  4.97,  5.83,  6.59,  5.14,  6.19,
        5.42,  5.49,  5.41,  5.08,  6.4 ,  6.12,  5.33,  5.8 ,  5.16,
        5.57,  5.62,  5.37,  5.42,  4.99,  5.29,  5.37,  5.04,  5.06,
        5.78,  6.03,  5.33,  6.44,  5.02,  5.17,  6.71,  5.73,  5.78,
        5.11,  5.02,  5.31,  6.  ,  5.38,  5.05,  5.3 ,  5.27,  6.48,
        5.63,  6.26,  6.43,  5.1 ,  5.98,  6.45,  6.42,  5.85,  5.83,
        5.89,  5.45,  6.29,  5.66,  5.73,  5.82,  6.76,  6.83,  5.41,
        6.77,  5.04,  5.47,  5.2 ,  6.47,  5.03,  4.75,  5.74,  4.99,
        5.62,  6.02,  5.85,  5.53,  6.06,  5.35,  5.01,  5.32,  5.94,
        5.16,  4.94,  5.93,  5.87,  5.09,  5.75,  6.14,  5.27,  5.33,
        5.36,  5.94,